In [ ]:
import xarray as xr
import intake
import pathlib
from zarr.convenience import consolidate_metadata

In [ ]:
from dask.distributed import Client

In [ ]:
client = Client()

In [ ]:
dir_scratch = pathlib.Path('/caldera/projects/usgs/hazards/cmgp/woodshole/rsignell/conus404')
file_chanobs = dir_scratch / 'chanobs.zarr'

In [ ]:
_ = consolidate_metadata(file_chanobs)

In [ ]:
ds_nwm_chrtout = xr.open_dataset(file_chanobs, engine='zarr', backend_kwargs=dict(consolidated=True), chunks={})

In [ ]:
gage_ids_str = ['USGS-'+gage_id.astype('str').lstrip() for gage_id in ds_nwm_chrtout['gage_id'].values]

In [ ]:
ds_nwm_chrtout = ds_nwm_chrtout.assign_coords({'site_id':xr.DataArray(gage_ids_str, dims='feature_id')})

In [ ]:
ds_nwm_chrtout = ds_nwm_chrtout.drop('gage_id')

In [ ]:
ds_nwm_chrtout = ds_nwm_chrtout.rename({'site_id':'gage_id'})

In [ ]:
ds_nwm_chrtout = ds_nwm_chrtout.drop('feature_id')

In [ ]:
ds_nwm_chrtout = ds_nwm_chrtout.rename({'gage_id':'feature_id'})

In [ ]:
ds_nwm_chrtout = ds_nwm_chrtout.rename_dims({'feature_id':'gage_id'}).rename({'feature_id':'gage_id'})

In [ ]:
ds_nwm_chrtout.to_zarr('/caldera/projects/usgs/hazards/cmgp/woodshole/rsignell/conus404/zarr/chanobs.zarr', 
                       mode='w', consolidated=True)

In [ ]:
client

In [ ]:
import intake

In [ ]:
url = 'https://raw.githubusercontent.com/nhm-usgs/data-pipeline-helpers/main/hytest/hytest_intake_catalog.yml'

In [ ]:
cat = intake.open_catalog(url)
list(cat)

In [ ]:
ds_nwis = cat['nwis-streamflow-on-prem'].to_dask()

In [ ]:
ds_nwis = ds_nwis.rename_dims({'feature_id':'gage_id'}).rename({'feature_id':'gage_id'})
ds_nwis

In [ ]:
ds_nwis.to_zarr('/caldera/projects/usgs/hazards/cmgp/woodshole/rsignell/conus404/zarr/nwis2_chanobs.zarr', 
                       mode='w', consolidated=True)